In [1]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# !pip install -U PyDrive

In [3]:
#Import authentication libraries

# from google.colab import auth
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

In [4]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [5]:
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [6]:
# # Auto-iterate through all files in the midi folder.
# file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
# for file1 in file_list:
#   print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

In [7]:
# # Download all midi files
# for file1 in file_list:
#   file_id=file1['id']
#   file_title = file1['title']
#   drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [8]:
# from os import listdir
# from os.path import isfile, join

In [9]:
# echo %cd%

In [10]:
# folder ='C:/Users/Gustavo/Documents/GitHub/MidiNet_test/cond2d'
# 

In [11]:
# songs=[f for f in listdir(folder)]
# print(songs)
# songs=songs[-10:-2]

In [12]:
# songs

In [13]:
# data_songs=np.empty((0,16,128), int)
# for song in songs:
#   song_opened = open(song,"rb")
#   song_loaded= pickle.load(song_opened)
#   #for track 1
#   for i in range(len(song_loaded[0])):
#     bar=np.empty((0,128), int)[None]
#     for j in song_loaded[0][i]:
#       note=np.zeros(128)[None][None]
#       if type(j) == type(1.5):
#         note[0,0,int(j)] = 1
#       bar=np.append(bar,note,axis=1)
#     data_songs =np.append(data_songs,bar, axis=0)
#   print(np.array(data_songs).shape)
# data_songs_prev =np.roll(data_songs,1,axis=0)
# data_songs=data_songs[None]
# data_songs_prev=data_songs_prev[None]

In [14]:
# print(data_songs.shape)
# print(data_songs_prev.shape)
# data_songs=np.transpose(data_songs, (1,0,2,3))#Real data, input for model
# data_songs_prev=np.transpose(data_songs_prev, (1,0,2,3))#Real data, 1 previous bar, input for model
# print(data_songs.shape)
# print(data_songs_prev.shape)

In [15]:
# np.save('data_songs.npy', data_songs, allow_pickle=True, fix_imports=True)

In [16]:
# np.save('data_songs_prev.npy', data_songs_prev, allow_pickle=True, fix_imports=True)

In [ ]:
import os
import scipy.misc
import numpy as np
from model import MidiNet #Llamando el modelo
from utils import pp, to_json, generation_test

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 20, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 12, "The size of batch [12]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Dircacectory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_vg", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_vg':
            model = MidiNet(sess, batch_size=FLAGS.batch_size, output_w=FLAGS.output_w, output_h=FLAGS.output_h, c_dim=FLAGS.c_dim,
                    dataset_name=FLAGS.dataset, is_crop=FLAGS.is_crop, checkpoint_dir=FLAGS.checkpoint_dir, sample_dir=FLAGS.sample_dir, 
                            gen_dir=FLAGS.gen_dir)
        
        if FLAGS.is_train:
            model.train(FLAGS)
        else:
            model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

In [ ]:
!tensorboard --logdir=log/